# Cleaning the [IFAR](http://www.ifar.org/catalogues_raisonnes.php?alpha=&searchtype=artist&published=1&inPrep=1&artist=&author=)

### Modules used throughout the document

The following packages and raw python is all you will need to clean data from IFAR.

In [71]:
import pandas as pd
import json
import re
import numpy as np

### Load your data

Once your spider finished collecting the data from IFAR, load it into memory using the JSON package and the syntax. You will be loading the data as a list full of python dictionaries. The first one at index 0 is the very first page we scraped, while the rest is a combination of the links to the published, and in-progress work of each artist.

In [72]:
# load the dataset
json_file = open('C:/Users/ramon/OneDrive/INSEAD/auction_proj/data.json', 'r')
data = json.load(json_file)

Check the types of first page and the rest.

In [75]:
type(data), type(data[0]), type(data[1])

(list, dict, dict)

### Cleaning the main page

Let's extract all of the variables in the main webpage first by selecting the first item in our list and then key for every column we are interested in (all of them).

```python
data[our first site]['the key we need']
```

In [76]:
# Extract the variables of the first page
name = data[0]['name'].copy()
links = data[0]['link'].copy()
birth_year = data[0]['birth_year'].copy()
birth_place = data[0]['birth_place'].copy()
death_year = data[0]['death_year'].copy()
death_place = data[0]['death_place'].copy()

Links are not immediately available with the full length required to be typed directly into a search bar so we'll get the full link by looping through each item in our links list, combining the first part of the link with the rest, and then appending the new values to a new list.

In [79]:
# Get the full links of the second pages before creating the dataframe
full_links = []
for link in links:
    full = 'http://www.ifar.org/' + link
    full_links.append(full)

# Check that the variables was created successfully by verifying the lenght and content
len(full_links), full_links[:2]

(2952,
 ['http://www.ifar.org/artist_name.php?nameid=1563&published=1&inPrep=1&author=',
  'http://www.ifar.org/artist_name.php?nameid=1396&published=1&inPrep=1&author='])

Let's create our first dataframe to see whether we need to do some further cleaning in the first page.

In [85]:
# Create the first dataset containing all of page 1
# Zip a list of lists for the data and pass in the column names we would like to use
page_one = pd.DataFrame(list(zip(name, full_links, birth_year, birth_place, death_year, death_place)),
                          columns = ['names', 'links', 'birth_year', 'birth_place', 'death_year', 'death_place'])

Inspect the first couple of rows of the dataset.

In [86]:
page_one.head()

,names,links,birth_year,birth_place,death_year,death_place
0,"Aachen, Hans von",http://www.ifar.org/artist_name.php?nameid=156...,"<td class=""g_center_birth_year"">1552</td>","<td class=""g_center_birth_place"">Germany</td>","<td class=""g_center_death_year"">1615</td>","<td class=""g_center_death_place"">Czech Republi..."
1,"Aalto, Alvar",http://www.ifar.org/artist_name.php?nameid=139...,"<td class=""g_center_birth_year"">1898</td>","<td class=""g_center_birth_place"">Finland</td>","<td class=""g_center_death_year"">1976</td>","<td class=""g_center_death_place"">Finland</td>"
2,"Abbati, Giuseppe",http://www.ifar.org/artist_name.php?nameid=139...,"<td class=""g_center_birth_year"">1836</td>","<td class=""g_center_birth_place"">Italy</td>","<td class=""g_center_death_year"">1868</td>","<td class=""g_center_death_place"">Italy</td>"
3,"Abdessemed, Adel",http://www.ifar.org/artist_name.php?nameid=307...,"<td class=""g_center_birth_year"">1971</td>","<td class=""g_center_birth_place"">Algeria</td>","<td class=""g_center_death_year""></td>","<td class=""g_center_death_place""></td>"
4,"Abelenda Zapata, Manuel",http://www.ifar.org/artist_name.php?nameid=203...,"<td class=""g_center_birth_year"">1889</td>","<td class=""g_center_birth_place"">Spain</td>","<td class=""g_center_death_year"">1957</td>","<td class=""g_center_death_place"">Spain</td>"


We need to get rid of the html code wrapped around the values we are interested in. Let's create a function for this.

In [92]:
def remove_html_tags(text):
    """
    Remove the html tags and keep the values you want.
    """
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

Now let's use the function we just created with every value in our columns, append the valus to a new list, and the substitute the list as a pandas Series in the respective columns of our dataframe.

In [93]:
# Clean all of the variables in the first page and add them again to the dataset

# Create the lists you will be appending the clean values to
clean_byear = []
clean_bplace = []
clean_dyear = []
clean_dplace = []

# Loop over each variable, clean the values, and append them to your new lists

for by in page_one['birth_year']:
    f = remove_html_tags(by)
    clean_byear.append(f)
    
for bp in page_one['birth_place']:
    q = remove_html_tags(bp)
    clean_bplace.append(q)
    
for dy in page_one['death_year']:
    e = remove_html_tags(dy)
    clean_dyear.append(e)
    
for dp in page_one['death_place']:
    w = remove_html_tags(dp)
    clean_dplace.append(w)

    
# Add your new lists to their respective variables in the dataframe

page_one['birth_year'] = pd.Series(clean_byear)
page_one['birth_place'] = pd.Series(clean_bplace)
page_one['death_year'] = pd.Series(clean_dyear)
page_one['death_place'] = pd.Series(clean_dplace)

Make sure that the variables were cleaned correctly.

In [94]:
page_one.head()

,names,links,birth_year,birth_place,death_year,death_place
0,"Aachen, Hans von",http://www.ifar.org/artist_name.php?nameid=156...,1552,Germany,1615,Czech Republic
1,"Aalto, Alvar",http://www.ifar.org/artist_name.php?nameid=139...,1898,Finland,1976,Finland
2,"Abbati, Giuseppe",http://www.ifar.org/artist_name.php?nameid=139...,1836,Italy,1868,Italy
3,"Abdessemed, Adel",http://www.ifar.org/artist_name.php?nameid=307...,1971,Algeria,,
4,"Abelenda Zapata, Manuel",http://www.ifar.org/artist_name.php?nameid=203...,1889,Spain,1957,Spain


### Second set of webpages

Make a copy of the rest of the dataset and exclude the first page.

In [95]:
second_part = data[1:].copy()

We have two kinds of dictionaries:
- One containing two key/value pairs that correspond to the data in the second page
- Another one with key/value pairs corresponding to the third page with the information we need

Let's inspect both of these.

In [98]:
second_part[1], second_part[100]

({'artist_one': 'Abelenda Zapata, Manuel',
  'box_info': ['\r\n                            ',
   'Published Catalogues',
   '\r\n\r\n                ',
   '\r\n\r\n                                                ',
   '\r\n                                ',
   'Manuel Abelenda : (1889-1957).                                    ',
   'click to learn more',
   '\r\n                            ',
   '\r\n                                                                    Adolfo de Abel Vilela                                                                    ',
   '\r\n                                ',
   '\r\n                                Fundacio´n Pedro Barrie de la Maza',
   '\r\n                                ',
   '\r\n                            ',
   '\r\n                            \r\n                        ',
   '\r\n                                                    ',
   '\r\n                                ',
   'Behnisch & Partner Bauten 1952-1992.                     

Let's now create our dataframe with the information in the second set of sites we crawled.

We will need to look over the dictionaries with only two key/value pairs. To do this we will first check whether the dictionary contains the unique key called _artist_one_ and if it does, we will append the the artist name and the _box_info_ values into two separate lists before creating the dataset.

In [142]:
# Create your lists

artist_one = []
box_info = []

# Go over all of the items in the second part and extract the information in the second pages

for i in second_part:
    if 'artist_one' in i.keys(): # Since we are interested in a matching value we apply the method .keys to each dictionary
        artist_one.append(i['artist_one']) # append the corresponding value to each variable
        box_info.append(i['box_info'])
    else:
        artist_one.append(np.nan) # nan if empty
        box_info.append('nan')

# Check that the lenght matches

len(artist_one), len(box_info)

(7961, 7961)

Now we can create the dataframe for our second set of websites.

In [143]:
page_two = pd.DataFrame(list(zip(artist_one, box_info)), columns=['artist_one', 'box_info'])
page_two.head()

,artist_one,box_info
0,"Aachen, Hans von","[\r\n , Published C..."
1,"Abelenda Zapata, Manuel","[\r\n , Published C..."
2,"Adams, Clinton","[\r\n , Published C..."
3,"Acht, René Charles","[\r\n , Published C..."
4,"Adam, Otto","[\r\n , Published C..."


Let's clean the box_info variable by looping over each item, stripping any form of whitespace, and then adding the new element to a list and the new list back to the dataframe.

In [144]:
# Clean the box

box_new = []

for i in page_two.box_info:
    tempora = []
    if i != 'nan':
        for s in i:
            gen = s.strip() # remove the whitespace
            tempora.append(gen) # append to new list
        join_them = list(filter(None, tempora)) # clear the blank elements from the box info
        box_new.append(join_them) # append clean list to list
    else:
        box_new.append(np.nan) # if element is null append nan to the list


len(box_new), box_new[59:100]

(7961,
 [['Published Catalogues',
   'Baldur Schönfelder Catalogue Raisonné.',
   'click to learn more',
   'Schäfer, Siegfried B.;&nbsp\r\n                                                                            Witteveen, Cecilia A.M.',
   'https://www.kunst-archive.net/en/'],
  ['Published Catalogues',
   'Richard Seewald. das Graphische Werk: Radierungen, Holzschnitte, Lithographien, Plakate, Linolschnitte',
   'click to learn more',
   'Jentsch, Ralph',
   'Esslingen : Verlag Kunstgalerie Esslingen, 1973.'],
  ['Published Catalogues',
   "Maurice Wyckaert : l'oeuvre peint, 1947-1996 : tentative de catalogue raisonné.",
   'click to learn more',
   'Berréby, Gérard;&nbsp\r\n                                                                            Orhan, Danielle',
   'Paris : Allia, 2012.'],
  ['Published Catalogues',
   'Erwin Wurm : One Minute Sculptures: Werkverzeichnis = Index of Works = catalogue raisonné 1988-1998.',
   'click to learn more',
   'Sans, Jérôme',
   'Brege

Now we have the completed the dataset for the second set of websites. There is further cleaning procedures we could take but since the variables of interest are in the following pages, we will leave the set of pages as is for now.

In [146]:
page_two['box_info'] = pd.Series(box_new)
page_two.head()

,artist_one,box_info
0,"Aachen, Hans von","[Published Catalogues, Hans von Aachen, 1552-1..."
1,"Abelenda Zapata, Manuel","[Published Catalogues, Manuel Abelenda : (1889..."
2,"Adams, Clinton","[Published Catalogues, A Meticulous Serenity :..."
3,"Acht, René Charles","[Published Catalogues, René Acht : Monographie..."
4,"Adam, Otto","[Published Catalogues, Der Maler Otto Adam (19..."


### Third set of web pages

In the same fashion we created we started the process for the second set of websites, let's clean the third one as well.

Create your blank lists, loop over the dictionaries and extract the values we need.

In [147]:
# Create your lists

artist_two = []
column_names = []
content = []

# Loop over each dictionary

for i in second_part:
    if 'artist_two' in i.keys(): # find the key in the dictionary
        artist_two.append(i['artist_two']) # match the corresponding values to each list 
        column_names.append(i['column_names'])
        content.append(i['content'])
    else:
        artist_two.append(np.nan) # if null append np.nan to your lists
        column_names.append(np.nan)
        content.append(np.nan)

# Check that the lenght matches
len(artist_two), len(column_names), len(content)

(7961, 7961, 7961)

In [200]:
page_three = pd.DataFrame(list(zip(artist_two, column_names, content)),
                         columns = ['artist', 'column_names', 'content'])
page_three.tail() # check the lenght and the content of the dataframe

,artist,column_names,content
7956,"Artist: Adler, Jankel","[Item Title:, Author:\r\n ...","[<dd>Jankel Adler (1895-1949). </dd>, <dd>\r\n..."
7957,"Artist: Adler, David A.","[Item Title:, Author(s):\r\n ...","[<dd>The Country Houses of David Adler. </dd>,..."
7958,"Artist: Adler, Dankmar","[Item Title:, Author(s):\r\n ...",[<dd>The Complete Architecture of Adler &amp; ...
7959,"Artist: Adams, Robert","[Item Title:, Author:\r\n ...","[<dd>The Sculpture of Robert Adams. </dd>, <dd..."
7960,"Artist: Adams, Mark","[Item Title:, Author(s):\r\n ...",[<dd>Mark Adams : Catalogue Raisonné of Tapest...


Now that we have our final dataset, let's get rid of the NaN values.

- First check if there are any NaN in the dataset
- Drop the rows where all values are NaN
- Reset your index

Notice that the reason why we have so many values in this dataset when compared to the amount of values in the main webpage is because we now have an artist for every work that was published or is still in-progress.

In [201]:
# First check if there are any
page_three.isna().sum()

artist          2974
column_names    2952
content         2952
dtype: int64

In [202]:
# Drop all of the NaN values
page_three = page_three.dropna(how='all')
page_three.head()

,artist,column_names,content
48,"Artist: Diziani, Gaspare","[Item Title:, Author:\r\n ...","[<dd>Gaspare Diziani. </dd>, <dd>\r\n ..."
49,"Artist: Dufy, Raoul","[Item Title:, Author(s):\r\n ...",[<dd>Raoul Dufy : catalogue raisonné de l'oeuv...
50,"Artist: Guidobono, Bartolomeo","[Item Title:, Author(s):\r\n ...","[<dd>Bartolomeo e Domenico Guidobono. </dd>, <..."
56,"Artist: Heath, Frederick","[Item Title:, Author:\r\n ...","[<dd>The Heath Family Engravers, 1779-1878. </..."
58,"Artist: Jones, John Llewelyn","[Item Title:, Author(s):\r\n ...",[<dd>John Llewelyn Jones : Australia's Forgott...


In [203]:
# Reset your index
page_three = page_three.reset_index(drop=True)
page_three.head()

,artist,column_names,content
0,"Artist: Diziani, Gaspare","[Item Title:, Author:\r\n ...","[<dd>Gaspare Diziani. </dd>, <dd>\r\n ..."
1,"Artist: Dufy, Raoul","[Item Title:, Author(s):\r\n ...",[<dd>Raoul Dufy : catalogue raisonné de l'oeuv...
2,"Artist: Guidobono, Bartolomeo","[Item Title:, Author(s):\r\n ...","[<dd>Bartolomeo e Domenico Guidobono. </dd>, <..."
3,"Artist: Heath, Frederick","[Item Title:, Author:\r\n ...","[<dd>The Heath Family Engravers, 1779-1878. </..."
4,"Artist: Jones, John Llewelyn","[Item Title:, Author(s):\r\n ...",[<dd>John Llewelyn Jones : Australia's Forgott...


If you noticed above when we checked for NaN values, our artist variable had more NaNs than the rest. Let's inspect those values to see if we missed something or if the data was simply unavailable.

In [204]:
page_three[page_three.artist.isna()]

,artist,column_names,content
164,None,[],[]
456,None,[],[]
478,None,[],[]
1053,None,[],[]
1772,None,[],[]
1962,None,[],[]
2029,None,[],[]
2560,None,[],[]
2698,None,[],[]
3194,None,[],[]


Clean the rest of the NaN values in the dataset

In [205]:
page_three = page_three[pd.notnull(page_three['artist'])]
page_three.head()

,artist,column_names,content
0,"Artist: Diziani, Gaspare","[Item Title:, Author:\r\n ...","[<dd>Gaspare Diziani. </dd>, <dd>\r\n ..."
1,"Artist: Dufy, Raoul","[Item Title:, Author(s):\r\n ...",[<dd>Raoul Dufy : catalogue raisonné de l'oeuv...
2,"Artist: Guidobono, Bartolomeo","[Item Title:, Author(s):\r\n ...","[<dd>Bartolomeo e Domenico Guidobono. </dd>, <..."
3,"Artist: Heath, Frederick","[Item Title:, Author:\r\n ...","[<dd>The Heath Family Engravers, 1779-1878. </..."
4,"Artist: Jones, John Llewelyn","[Item Title:, Author(s):\r\n ...",[<dd>John Llewelyn Jones : Australia's Forgott...


Reset the index again.

In [206]:
page_three = page_three.reset_index(drop=True)
page_three.head()

,artist,column_names,content
0,"Artist: Diziani, Gaspare","[Item Title:, Author:\r\n ...","[<dd>Gaspare Diziani. </dd>, <dd>\r\n ..."
1,"Artist: Dufy, Raoul","[Item Title:, Author(s):\r\n ...",[<dd>Raoul Dufy : catalogue raisonné de l'oeuv...
2,"Artist: Guidobono, Bartolomeo","[Item Title:, Author(s):\r\n ...","[<dd>Bartolomeo e Domenico Guidobono. </dd>, <..."
3,"Artist: Heath, Frederick","[Item Title:, Author:\r\n ...","[<dd>The Heath Family Engravers, 1779-1878. </..."
4,"Artist: Jones, John Llewelyn","[Item Title:, Author(s):\r\n ...",[<dd>John Llewelyn Jones : Australia's Forgott...


In [213]:
len(page_three)

4987

In [214]:
sum(page_three.isna().any())

0

We now have all of the values we need but we still need to take care of a few things. We strill need to:

- Take the 'Artist: ' out of the artist column
- Split the column names and make them our variables
- Clean the content variable


Let's continue by taking the 'Artist: ' out of our artist variable.

1. Create an empty list
2. Find the word in the string
3. Take it out and create a sublist
4. Clean the sublist
5. Append the artist name to the new list

In [218]:
artist_three = []
for i in page_three.artist:
    if 'Artist: ' in str(i): # Check if the string has 'Artist: ' in it
        gen = i.split('Artist: ') # Separate it from the string
        artist_three.append(gen[1].strip()) # Append the cleaned artist name to our new list 
    else:
        artist_three.append(np.nan)
    
len(artist_three), artist_three[:10]

(4987,
 ['Diziani, Gaspare',
  'Dufy, Raoul',
  'Guidobono, Bartolomeo',
  'Heath, Frederick',
  'Jones, John Llewelyn',
  'Moy, Seong',
  'Schönebeck, Eugen',
  'Paresce, Renato',
  'Patel, Jacques',
  'Schönebeck, Eugen'])

Let's do a similar process with the other two columns.

In [219]:
# Clean the new columns
cols_news = []
for i in page_three.column_names: # For every list in our column
    tempora = []
    if i != np.nan: # Althought there are no nans in the dataset it can pay off in the future to be extra cautious
        for s in i: # For every element in our list
            gen = s.strip() # Clean spaces and assign a variable to it
            tempora.append(gen) # Attach it to a temporary list
        cols_news.append(tempora) # Attach it to our new col
    else:
        cols_news.append(np.nan)
    
len(cols_news), cols_news[:3]

(4987,
 [['Item Title:',
   'Author:',
   'Imprint:',
   'Language:',
   'Size:',
   'Pages:',
   'Illustrations:',
   'Concordance:',
   'Bibliography:',
   'Index:',
   'Exhibition List:',
   'Chronology:',
   'Content Note:',
   'Individual Entries Contain:',
   'Public Note:'],
  ['Item Title:',
   'Author(s):',
   'ISBN:',
   'Item Number:',
   'Imprint:',
   'Language:',
   'Size:',
   'Pages:',
   'Illustrations:',
   'Concordance:',
   'Bibliography:',
   'Index:',
   'Exhibition List:',
   'Chronology:',
   'Content Note:',
   'Individual Entries Contain:',
   'Public Note:'],
  ['Item Title:',
   'Author(s):',
   'ISBN:',
   'Imprint:',
   'Language(s):',
   'Size:',
   'Pages:',
   'Illustrations:',
   'Concordance:',
   'Bibliography:',
   'Index:',
   'Exhibition List:',
   'Chronology:',
   'Content Note:',
   'Individual Entries Contain:',
   'Public Note:']])

In [220]:
# Create your new list
new_content = []

for i in page_three.content: # loop over the variable of interest
    tempora = [] # create a temporary list
    if i != np.nan: # If the element is not a NaN
        for s in i: # Select each element
            gen = remove_html_tags(s) # Remove the tags in it
            sus = gen.strip() # Clear any whitespace
            tempora.append(sus) # Attach it to our new list
        join_them = list(filter(None, tempora)) # Filter out empty lists
        new_content.append(tempora) # Append to our final list
    else:
        new_content.append(np.nan)
    
len(new_content), new_content[:3]

(4987,
 [['Gaspare Diziani.',
   'Zugni-Tauro, Anna Paola',
   'Venice : Alfieri, 1971.',
   'Italian',
   '28 cm',
   '363',
   'numerous ills., 126 plates (16 mounted)',
   'No',
   'Yes',
   'No',
   'No',
   'No',
   'The unillustrated catalogue raisonné (with cross references to the plates and illustrations within the text) is organized by location or collection; it gives title, medium and support, dating information, dimensions, notes, and bibliographical references. \nThe organization of the lists is topographical, and by classification: works of religious character, civil commissions, and private collectors.',
   'Bibliog.; Comments',
   "Introductory material includes a biography, a discussion of the artist's critical fortunes and other essays. The catalogue raisonné presents the works as either autograph, attributed to Diziani, by one of the artist's followers of interest to historians, and lost/unlocated. There are also appendices discusssing and cataloging  Diziani's two so

In [221]:
double_list = list(zip(cols_news, new_content))
double_list[100][1]

['Rik Wouters : catalogue raisonné = les peintures = de schilderijen.',
 'Bertrand, Olivier                                                            ;&amp;nbsp\r\n                                                            Cockshaw, Pierre (preface)',
 '9789053250433',
 'Antwerp : Petraco-Pandora, 1995.',
 'French                                                            ;&amp;nbsp\r\n                                                            Dutch',
 '30 cm',
 '264',
 'b&amp;w, some color',
 'No',
 'Yes',
 'Yes',
 'No',
 'No',
 'The publication includes an alphabetical list of the paintings.']

Publications and work in process both have different variables in them. To make sure we are looking at the correct one, we will loop over each variable, check whether an element exists, and classify it as:

- 1 for Publications
- 2 for work in progress

In [222]:
pub_or_prep = []
for i in cols_news:
    if 'Item Title:' in i:
        pub_or_prep.append(1)
    else:
        pub_or_prep.append(2)        
    
pub_or_prep[-25:]

[1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Let's now combine our new variables into a dataframe. Remember that our column names and the content are now zipped together.

In [223]:
dos_clean = pd.DataFrame(list(zip(artist_three, pub_or_prep, double_list)),
                         columns = ['artist', 'pub_or_prep', 'col_names_content'])

dos_clean.head()

,artist,pub_or_prep,col_names_content
0,"Diziani, Gaspare",1,"([Item Title:, Author:, Imprint:, Language:, S..."
1,"Dufy, Raoul",1,"([Item Title:, Author(s):, ISBN:, Item Number:..."
2,"Guidobono, Bartolomeo",1,"([Item Title:, Author(s):, ISBN:, Imprint:, La..."
3,"Heath, Frederick",1,"([Item Title:, Author:, ISBN:, Item Number:, I..."
4,"Jones, John Llewelyn",1,"([Item Title:, Author(s):, ISBN:, Imprint:, La..."


### Final cleaning
In order to take the columns out and match an element of the columns with an element of the content, we'll need to create a function for this to match the indext of one with the value of the other.

We'll need to pass through it three things:

- A Series or vector
- The name of the column
- The empty list


In [224]:
def gen_columns(series, name, new_list): # arguments of our function
    for i in series: # go through the series
        if name in i[0]: # if the name of a variable is in the first list of an element in our series
            new_list.append(i[1][i[0].index(name)]) # take the index of that element and take a matching value from the second list of that element
        else:
            new_list.append(np.nan)

In [225]:
# Create empty lists for our variables
artist = []
item_t = []
author = []
author_s = []
isbn = []
imprint = []
language = []
size = []
pages = []
illustrations = []
concordance = []
bibliography = []
index = []
exhibition_list = []
cronology = []
ind_entr_cont = []
public_note = []


# Apply the functions with the required arguments

gen_columns(series = dos_clean.col_names_content, name = 'Artist:', new_list = artist)
gen_columns(series = dos_clean.col_names_content, name = 'Item Title:', new_list = item_t)
gen_columns(series = dos_clean.col_names_content, name = 'Author:', new_list = author)
gen_columns(series = dos_clean.col_names_content, name = 'Author(s):', new_list = author_s)
gen_columns(series = dos_clean.col_names_content, name = 'ISBN:', new_list = isbn)
gen_columns(series = dos_clean.col_names_content, name = 'Imprint:', new_list = imprint)
gen_columns(series = dos_clean.col_names_content, name = 'Language:', new_list = language)
gen_columns(series = dos_clean.col_names_content, name = 'Size:', new_list = size)
gen_columns(series = dos_clean.col_names_content, name = 'Pages:', new_list = pages)
gen_columns(series = dos_clean.col_names_content, name = 'Illustrations:', new_list = illustrations)
gen_columns(series = dos_clean.col_names_content, name = 'Concordance:', new_list = concordance)
gen_columns(series = dos_clean.col_names_content, name = 'Bibliography:', new_list = bibliography)
gen_columns(series = dos_clean.col_names_content, name = 'Index:', new_list = index)
gen_columns(series = dos_clean.col_names_content, name = 'Exhibition List:', new_list = exhibition_list)
gen_columns(series = dos_clean.col_names_content, name = 'Chronology:', new_list = cronology)
gen_columns(series = dos_clean.col_names_content, name = 'Individual Entries Contain:', new_list = ind_entr_cont)
gen_columns(series = dos_clean.col_names_content, name = 'Public Note:', new_list = public_note)

Let's check that the variables were created correctly by checking the content of our new lists.

In [226]:
artist[:3], item_t[:3], author[:3], author_s[:3], isbn[:3], imprint[:3]

([nan, nan, nan],
 ['Gaspare Diziani.',
  "Raoul Dufy : catalogue raisonné de l'oeuvre peint. Supplément.",
  'Bartolomeo e Domenico Guidobono.'],
 ['Zugni-Tauro, Anna Paola', nan, nan],
 [nan,
  'Guillon-Laffaille, Fanny (bibliog.)                                                            ;&amp;nbsp\r\n                                                            Laffaille, Maurice',
  'Newcome Schleier, Mary                                                            ;&amp;nbsp\r\n                                                            Cameirana, Arrigo                                                            ;&amp;nbsp\r\n                                                            Orlando, Anna'],
 [nan, '2865740056', '8880520156'],
 ['Venice : Alfieri, 1971.',
  'Paris : Éditions Louis Carré, 1985.',
  'Turin : Artema : Compagnia di belle arti, 2002.'])

In [227]:
language[:3], size[:3], pages[:3], illustrations[:3], concordance[:3], bibliography[:3]

(['Italian', 'French', nan],
 ['28 cm', '29 cm', '30 cm'],
 ['363', '212', 'xxvii, 217'],
 ['numerous ills., 126 plates (16 mounted)',
  '243 b&amp;w, 39 color',
  'b&amp;w, some color'],
 ['No', 'No', 'No'],
 ['Yes', 'Yes', 'Yes'])

In [228]:
index[:3], exhibition_list[:3], cronology[:3], ind_entr_cont[:3]

(['No', 'Yes', 'Yes'],
 ['No', 'No', 'No'],
 ['No', 'No', 'No'],
 ['Bibliog.; Comments',
  'Exhib. Hist; Bibliog.; Comments',
  'Provenance; Bibliog.; Comments'])

Create a copy of the dataset.

In [257]:
df_final = dos_clean.copy()
df_final.head()

,artist,pub_or_prep,col_names_content
0,"Diziani, Gaspare",1,"([Item Title:, Author:, Imprint:, Language:, S..."
1,"Dufy, Raoul",1,"([Item Title:, Author(s):, ISBN:, Item Number:..."
2,"Guidobono, Bartolomeo",1,"([Item Title:, Author(s):, ISBN:, Imprint:, La..."
3,"Heath, Frederick",1,"([Item Title:, Author:, ISBN:, Item Number:, I..."
4,"Jones, John Llewelyn",1,"([Item Title:, Author(s):, ISBN:, Imprint:, La..."


Drop the combined variable, col_names_content, and then add the new variables to the dataframe.

In [258]:
df_final = df_final.drop(columns='col_names_content')

In [259]:
df_final.head(10)

,artist,pub_or_prep
0,"Diziani, Gaspare",1
1,"Dufy, Raoul",1
2,"Guidobono, Bartolomeo",1
3,"Heath, Frederick",1
4,"Jones, John Llewelyn",1
5,"Moy, Seong",2
6,"Schönebeck, Eugen",2
7,"Paresce, Renato",1
8,"Patel, Jacques",1
9,"Schönebeck, Eugen",1


In [261]:
df_final['artist_2'] = pd.Series(artist)
df_final['item_t'] = pd.Series(item_t)
df_final['author'] = pd.Series(author)
df_final['author_s'] = pd.Series(author_s)
df_final['isbn'] = pd.Series(isbn)
df_final['imprint'] = pd.Series(imprint)
df_final['language'] = pd.Series(language)
df_final['size'] = pd.Series(size)
df_final['pages'] = pd.Series(pages)
df_final['illustrations'] = pd.Series(illustrations)
df_final['concordance'] = pd.Series(concordance)
df_final['bibliography'] = pd.Series(bibliography)
df_final['index'] = pd.Series(index)
df_final['exhibition_list'] = pd.Series(exhibition_list)
df_final['cronology'] = pd.Series(cronology)
df_final['ind_entr_cont'] = pd.Series(ind_entr_cont)
df_final['public_note'] = pd.Series(public_note)

Since the variable artist_2 repeats elements in the artist column, let's get rid of it.

In [262]:
df_final = df_final.drop(columns='artist_2')
df_final.head()

,artist,pub_or_prep,item_t,author,author_s,isbn,imprint,language,size,pages,illustrations,concordance,bibliography,index,exhibition_list,cronology,ind_entr_cont,public_note
0,"Diziani, Gaspare",1,Gaspare Diziani.,"Zugni-Tauro, Anna Paola",NaN,NaN,"Venice : Alfieri, 1971.",Italian,28 cm,363,"numerous ills., 126 plates (16 mounted)",No,Yes,No,No,No,Bibliog.; Comments,"Introductory material includes a biography, a ..."
1,"Dufy, Raoul",1,Raoul Dufy : catalogue raisonné de l'oeuvre pe...,NaN,"Guillon-Laffaille, Fanny (bibliog.) ...",2865740056,"Paris : Éditions Louis Carré, 1985.",French,29 cm,212,"243 b&amp;w, 39 color",No,Yes,Yes,No,No,Exhib. Hist; Bibliog.; Comments,This is considered the fifth volume of the cat...
2,"Guidobono, Bartolomeo",1,Bartolomeo e Domenico Guidobono.,NaN,"Newcome Schleier, Mary ...",8880520156,"Turin : Artema : Compagnia di belle arti, 2002.",NaN,30 cm,"xxvii, 217","b&amp;w, some color",No,Yes,Yes,No,No,Provenance; Bibliog.; Comments,This study covers painted works found in churc...
3,"Heath, Frederick",1,"The Heath Family Engravers, 1779-1878.","Heath, John",NaN,085967908X,"Aldershot : Scolar Press, 1993.",English,26 cm,v. 1: 242; v. 2: 351,40 b&amp;w ills.,No,Yes,Yes,No,No,NaN,Each volume begins with sections on the histor...
4,"Jones, John Llewelyn",1,John Llewelyn Jones : Australia's Forgotten Pa...,NaN,"Aufy, Giles ...",0646348868,"Edgecliff, N.S.W. : Corbally Stourton Contempo...",English,30 cm,279,color,No,Yes,No,No,No,NaN,NaN


In [264]:
len(df_final.author_s)

4987

The last bit of cleaning we will do is with the columns containig multiple authors and languages. We will first split the elements in the columns by their delimiter ';'. Then we will strip any whitespace in between them and proceed to append them to a new list.

In [265]:
# Create a new list

new_auths = []


for i in df_final.author_s: # loop over every element of the variable
    tmp = []
    if pd.notna(i) == True: # if the element is not NaN
        val = i.split(';') # Split it by ';'
        for s in val: # For those variables we just splitted
            val_2 = s.strip().strip('&amp').strip('nbsp\r\n').strip() # clean them
            tmp.append(val_2) # append them to a temporary list
        join_them = list(filter(None, tmp)) # filter out the empty elements
        new_auths.append(join_them) # append back to main list
    else:
        new_auths.append(np.nan)
        
len(new_auths), new_auths[:3]

(4987,
 [nan,
  ['Guillon-Laffaille, Fanny (bibliog.)', 'Laffaille, Maurice'],
  ['Newcome Schleier, Mary', 'Cameirana, Arrigo', 'Orlando, A']])

Since we don't want a list with only one element, we will add an additional step so that we only need to append one string when the list has only one element and NaN when it has non.

In [266]:
# Create a new list

new_lang = []


for i in df_final.language:
    tmp = []
    if pd.notna(i) == True:
        val = i.split(';')
        for s in val:
            val_2 = s.strip()#.strip('&amp').strip('nbspr\n').strip()
            tmp.append(val_2)
        join_them = list(filter(None, tmp))
        #new_lang.append(join_them)
        if len(join_them) == 1:
            new_lang.append(join_them[0])
        else:
            new_lang.append(join_them[0])
    else:
        new_lang.append(np.nan)
        
len(new_lang), new_lang[:20]

(4987,
 ['Italian',
  'French',
  nan,
  'English',
  'English',
  nan,
  nan,
  nan,
  'French',
  'German',
  nan,
  'German',
  'French',
  nan,
  'French',
  'French',
  'German',
  nan,
  'Italian',
  'Spanish'])

In [268]:
# Add the the new variables to the dataset
df_final = df_final.drop(columns=['author_s', 'language'])
df_final.insert(4, 'author_s', new_auths, True)
df_final.insert(7, 'language', new_lang, True)
df_final.head()

,artist,pub_or_prep,item_t,author,author_s,isbn,imprint,language,size,pages,illustrations,concordance,bibliography,index,exhibition_list,cronology,ind_entr_cont,public_note
0,"Diziani, Gaspare",1,Gaspare Diziani.,"Zugni-Tauro, Anna Paola",NaN,NaN,"Venice : Alfieri, 1971.",Italian,28 cm,363,"numerous ills., 126 plates (16 mounted)",No,Yes,No,No,No,Bibliog.; Comments,"Introductory material includes a biography, a ..."
1,"Dufy, Raoul",1,Raoul Dufy : catalogue raisonné de l'oeuvre pe...,NaN,"[Guillon-Laffaille, Fanny (bibliog.), Laffaill...",2865740056,"Paris : Éditions Louis Carré, 1985.",French,29 cm,212,"243 b&amp;w, 39 color",No,Yes,Yes,No,No,Exhib. Hist; Bibliog.; Comments,This is considered the fifth volume of the cat...
2,"Guidobono, Bartolomeo",1,Bartolomeo e Domenico Guidobono.,NaN,"[Newcome Schleier, Mary, Cameirana, Arrigo, Or...",8880520156,"Turin : Artema : Compagnia di belle arti, 2002.",NaN,30 cm,"xxvii, 217","b&amp;w, some color",No,Yes,Yes,No,No,Provenance; Bibliog.; Comments,This study covers painted works found in churc...
3,"Heath, Frederick",1,"The Heath Family Engravers, 1779-1878.","Heath, John",NaN,085967908X,"Aldershot : Scolar Press, 1993.",English,26 cm,v. 1: 242; v. 2: 351,40 b&amp;w ills.,No,Yes,Yes,No,No,NaN,Each volume begins with sections on the histor...
4,"Jones, John Llewelyn",1,John Llewelyn Jones : Australia's Forgotten Pa...,NaN,"[Aufy, Gile, Corbally Stourton, Patrick]",0646348868,"Edgecliff, N.S.W. : Corbally Stourton Contempo...",English,30 cm,279,color,No,Yes,No,No,No,NaN,NaN


### Save the dataset

To save our new dataset we will use UTF-8 encoding to account with the different spellings encountered throughout the data. We can do this by adding as an argument the **encoding='utf-8'** option in pd.to_csv().

In [269]:
df_final.to_csv('C:/Users/ramon/OneDrive/INSEAD/auction_proj/full_data.csv', index=False, encoding='utf-8')

In [270]:
df = pd.read_csv('C:/Users/ramon/OneDrive/INSEAD/auction_proj/full_data.csv')

Let's check that the saving the data was done correctly.

In [271]:
len(df.columns), len(df)

(18, 4987)

In [272]:
df.head()

,artist,pub_or_prep,item_t,author,author_s,isbn,imprint,language,size,pages,illustrations,concordance,bibliography,index,exhibition_list,cronology,ind_entr_cont,public_note
0,"Diziani, Gaspare",1,Gaspare Diziani.,"Zugni-Tauro, Anna Paola",NaN,NaN,"Venice : Alfieri, 1971.",Italian,28 cm,363,"numerous ills., 126 plates (16 mounted)",No,Yes,No,No,No,Bibliog.; Comments,"Introductory material includes a biography, a ..."
1,"Dufy, Raoul",1,Raoul Dufy : catalogue raisonné de l'oeuvre pe...,NaN,"['Guillon-Laffaille, Fanny (bibliog.)', 'Laffa...",2865740056,"Paris : Éditions Louis Carré, 1985.",French,29 cm,212,"243 b&amp;w, 39 color",No,Yes,Yes,No,No,Exhib. Hist; Bibliog.; Comments,This is considered the fifth volume of the cat...
2,"Guidobono, Bartolomeo",1,Bartolomeo e Domenico Guidobono.,NaN,"['Newcome Schleier, Mary', 'Cameirana, Arrigo'...",8880520156,"Turin : Artema : Compagnia di belle arti, 2002.",NaN,30 cm,"xxvii, 217","b&amp;w, some color",No,Yes,Yes,No,No,Provenance; Bibliog.; Comments,This study covers painted works found in churc...
3,"Heath, Frederick",1,"The Heath Family Engravers, 1779-1878.","Heath, John",NaN,085967908X,"Aldershot : Scolar Press, 1993.",English,26 cm,v. 1: 242; v. 2: 351,40 b&amp;w ills.,No,Yes,Yes,No,No,NaN,Each volume begins with sections on the histor...
4,"Jones, John Llewelyn",1,John Llewelyn Jones : Australia's Forgotten Pa...,NaN,"['Aufy, Gile', 'Corbally Stourton, Patrick']",0646348868,"Edgecliff, N.S.W. : Corbally Stourton Contempo...",English,30 cm,279,color,No,Yes,No,No,No,NaN,NaN
